for params: n_estimators: 150
    max_depth: 13
    min_samples_leaf: 4
    min_samples_split: 2
    max_features: "sqrt"
    random_state: 50

Scorings for rf are: roc_auc: 0.9469485705656256, f1: 0.8362102171979072, precision: 0.8356205688942357, recall: 0.8444415474366913, accuracy: 0.8444415474366913

In [2]:
import pickle 
import pandas as pd

C:\Users\voqua\AppData\Local\Temp\ipykernel_7740\2193490567.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# procressed input data - combine employment into one and drop unsatisfied columns
with open('..\outputs\shap_values\shap_explanation.pkl', 'rb') as f:
    shap_rf = pickle.load(f)

<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
C:\Users\voqua\AppData\Local\Temp\ipykernel_7740\1315121445.py:2: SyntaxWarning: invalid escape sequence '\o'
  with open('..\outputs\shap_values\shap_explanation.pkl', 'rb') as f:


In [4]:
file_path = '../outputs/datasets/preprocessed_ds.csv'
preprocessed_ds = pd.read_csv(file_path)

In [5]:
preprocessed_ds

,age,num_previous_recidivisms,race_Caucasian,race_Hispanic,gender_Male,HighestEducationLevel_High_School_Diploma,HighestEducationLevel_No_HS_Diploma,HighestEducationLevel_Some_College,c_vio,placement,Employed,weighted_jl_total,weighted_wr_total,weighted_hd_total
0,46,0,1,0,1,1,0,0,0.1,0,1,0.01197,0.015408,0.00054
1,34,0,1,0,0,1,0,0,0.1,0,1,0.01197,0.015480,0.00054
2,36,0,1,0,1,0,0,1,0.1,0,1,0.01197,0.015552,0.00054
3,44,0,1,0,1,1,0,0,0.1,0,1,0.01197,0.015624,0.00054
4,31,0,1,0,1,0,1,0,0.1,0,1,0.01197,0.015696,0.00054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,32,2,1,0,0,0,1,0,0.5,2,0,0.35568,0.157248,0.13608
102270,32,0,1,0,0,0,0,1,0.5,0,1,0.35712,0.157248,0.13608
102271,36,0,1,0,1,0,1,0,0.5,0,1,0.35712,0.157680,0.13608
102272,35,0,1,0,0,1,0,0,0.5,0,1,0.35712,0.158112,0.13608


In [6]:
def preprocess_data(continuous_after: pd.DataFrame) -> pd.DataFrame:
    unknown_employment_index = continuous_after[ (continuous_after['employmentStatus_Full_Time'] == 0) & (continuous_after['employmentStatus_Part_Time'] == 0) & (continuous_after['employmentStatus_Unemployed'] == 0)].index
    unknown_employment_age = continuous_after[ (continuous_after['employmentStatus_Part_Time'] == 0) & (continuous_after['employmentStatus_Unemployed'] == 0) & (continuous_after['employmentStatus_Full_Time'] == 0) ]['age']
    unemployment_index = continuous_after[ continuous_after['employmentStatus_Unemployed'] == 1].index
    unknown_employment_index

    type1_index = list(unemployment_index) + list(unknown_employment_index)
    type2_index = list(continuous_after[ continuous_after['employmentStatus_Full_Time'] == 1].index) + list(continuous_after[ continuous_after['employmentStatus_Part_Time'] == 1].index)
    new_row = [0] * continuous_after.shape[0]
    for index in type1_index:
        new_row[index] = 1

    continuous_after['Employed'] = new_row
    continuous_after.drop(['employmentStatus_Part_Time', 'employmentStatus_Full_Time'], axis =1, inplace = True )

    new_row = [0] * continuous_after.shape[0]
    for index in type2_index:
        new_row[index] = 1
    
    continuous_after['Unemployed'] = new_row
    continuous_after.drop(['employmentStatus_Unemployed'], axis =1, inplace = True )
    continuous_after.drop(['Unemployed','ocp_wr_low_los', 'ocp_jl_low_los', 'ocp_hd_low_los','licenseStatus_Not_Suspended','licenseStatus_Suspended'], axis=1, inplace=True) 

    #Cost multiplication
    continuous_after['weighted_jl_total'] = continuous_after['c_occu_jail']*continuous_after['ocp_jl_total']
    continuous_after['weighted_wr_total'] = continuous_after['c_occu_wr']*continuous_after['ocp_wr_total']
    continuous_after['weighted_hd_total'] = continuous_after['c_occu_hd']*continuous_after['ocp_hd_total']
    continuous_after.drop(['c_occu_jail', 'c_occu_wr', 'c_occu_hd', 'ocp_jl_total', 'ocp_wr_total', 'ocp_hd_total'], axis = 1, inplace=True)

    # one hot encoding  -> the target is "placement", so we don't need to encode this column, also bcz RandomForest can handle categorical target data
    # work release = 0 | home detention = 1 | jail = 2
    for ind, item in continuous_after.iterrows():
        if item['placement'] == 'work release':
            continuous_after['placement'].iat[ind] = '0'
        elif item['placement'] == 'home detention':
            continuous_after['placement'].iat[ind] = '1'
        elif item['placement'] == 'jail':
            continuous_after['placement'].iat[ind] = '2'

    #object to int
    continuous_after['placement'] = pd.to_numeric(continuous_after['placement'])

    # drop imbalanced data #JUNE 5th: TUNR OFF THESE DROPPINGS
    #continuous_after.drop(['race_Other', 'registeredSexOffender_TRUE', 'violentOffender_TRUE', 'gangMember_x_TRUE', 'homeless_TRUE'], axis = 1, inplace = True)

    return continuous_after 

In [7]:
from sklearn.model_selection import train_test_split
def split_data(my_csv: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    y = my_csv["placement"]
    X = my_csv.drop(["placement"], axis = 1)

    X_train, X_test, y_train, y_test = train_test_split( X, y, train_size = 0.7, random_state = 50) 

    return X_train, X_test, y_train, y_test

In [8]:
file_path = '../outputs/datasets/demographics_to_placement_simulation_WR_no_cap_continuous_2.csv'
preprocessed_ds = pd.read_csv(file_path)

In [9]:
preprocessed_ds.columns

Index(['age', 'num_previous_recidivisms', 'race_Caucasian', 'race_Hispanic',
       'race_Other', 'gender_Male', 'registeredSexOffender_TRUE',
       'violentOffender_TRUE', 'gangMember_x_TRUE', 'homeless_TRUE',
       'employmentStatus_Full_Time', 'employmentStatus_Part_Time',
       'employmentStatus_Unemployed',
       'HighestEducationLevel_High_School_Diploma',
       'HighestEducationLevel_No_HS_Diploma',
       'HighestEducationLevel_Some_College', 'licenseStatus_Not_Suspended',
       'licenseStatus_Suspended', 'c_occu_jail', 'c_occu_wr', 'c_occu_hd',
       'c_vio', 'ocp_jl_low_los', 'ocp_jl_total', 'ocp_wr_low_los',
       'ocp_wr_total', 'ocp_hd_low_los', 'ocp_hd_total', 'placement'],
      dtype='object')

In [ ]:
import lime
import lime.lime_tabular

: 